# Importar librerías

In [1]:
import os
import requests
import pandas as pd

In [2]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [4]:
import pickle

# Crear directorio

En este directorio se guardará el dataset obtenido.

In [5]:
directory = "dataset"
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(parent_dir)

path = os.path.join(parent_dir, directory)

try:
    os.mkdir(path)
    print(f"Se ha creado el directorio {directory} en: {parent_dir}")
except OSError as error:
    print(f"El directorio {directory} en: {parent_dir} ya existe.")

g:\Proyectos_Github\fuel-predict\docker
El directorio dataset en: g:\Proyectos_Github\fuel-predict\docker ya existe.


# Obtención del dataset

Es equivalente a lo siguiente: `$.result.results[0].resources[1].url`

Donde `[1]` es para el csv y `[2]` para el json. Para trabajar con pandas prefiero el csv.

In [6]:
header = {"User-Agent": "Application"}
dataset = "raw_dataset.csv"
csv_path = os.path.join(path, dataset)
year = 2016

while True:
    url = f"https://catalogodatos.cnmc.es/api/3/action/package_search?q=Precios%20diarios%20provinciales%20-%20{year}%20-"
    r = requests.get(url, headers=header)

    if r.status_code == 200:
        data_catalog = r.json()
    
        if data_catalog["result"]["results"]:
            id_resource = data_catalog["result"]["results"][0]["resources"][1]["url"]
            downloaded_file = requests.get(id_resource)
    
            with open(csv_path, "ab") as f:
                f.write(downloaded_file.content)
    
            print(f"Descargados los datos para el año {year}.")
            year += 1
        else:
            print("No hay más datos que descargar.")
            break
    else:
        print(f"No ha podido procesarse la solicitud. Código: {r.status_code}")
        break

Descargados los datos para el año 2016.
Descargados los datos para el año 2017.
Descargados los datos para el año 2018.
Descargados los datos para el año 2019.
Descargados los datos para el año 2020.
Descargados los datos para el año 2021.
Descargados los datos para el año 2022.
Descargados los datos para el año 2023.
Descargados los datos para el año 2024.
Descargados los datos para el año 2025.
No hay más datos que descargar.


# Dataframe

## Lectura y visualizado

In [7]:
df_fuel = pd.read_csv(csv_path, delimiter=';')
df_fuel.head()

,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro
0,2016-01-01,Albacete,Gasolina 95 E5,"0,482","1,155"
1,2016-01-01,Albacete,Gasolina 98 E5,"0,552","1,278"
2,2016-01-01,Albacete,Gasóleo A habitual,"0,441","0,993"
3,2016-01-01,Albacete,Gasóleo Premium,"0,504","1,069"
4,2016-01-01,Alicante/Alacant,Gasolina 95 E5,"0,483","1,157"


In [8]:
rows, cols = df_fuel.shape
print(f"Filas: {rows} | Columnas: {cols}")

Filas: 687040 | Columnas: 5


## Comprobación de nulos y tipo de datos

In [9]:
df_fuel.isna().sum()

Fecha Precio                           0
Provincia                              0
Producto                               0
Promedio de Pai Diario CUBO €/litro    0
Promedio de Pvp Diario CUBO €/litro    0
dtype: int64

¡No hay nulos!

In [10]:
df_fuel.dtypes

Fecha Precio                           object
Provincia                              object
Producto                               object
Promedio de Pai Diario CUBO €/litro    object
Promedio de Pvp Diario CUBO €/litro    object
dtype: object

## Provincias

Queremos saber qué provincias hay en el dataset:

In [11]:
df_fuel["Provincia"].unique()

array(['Albacete', 'Alicante/Alacant', 'Almería', 'Araba/Álava',
       'Asturias', 'Ávila', 'Badajoz', 'Balears, Illes', 'Barcelona',
       'Bizkaia', 'Burgos', 'Cáceres', 'Cádiz', 'Cantabria',
       'Castellón/Castelló', 'Ceuta', 'Ciudad Real', 'Córdoba',
       'Coruña, A', 'Cuenca', 'Gipuzkoa', 'Girona', 'Granada',
       'Guadalajara', 'Huelva', 'Huesca', 'Jaén', 'León', 'Lleida',
       'Lugo', 'Madrid', 'Málaga', 'Melilla', 'Murcia', 'Navarra',
       'Ourense', 'Palencia', 'Palmas, Las', 'Pontevedra', 'Rioja, La',
       'Salamanca', 'Santa Cruz de Tenerife', 'Segovia', 'Sevilla',
       'Soria', 'Tarragona', 'Teruel', 'Toledo', 'Valencia/València',
       'Valladolid', 'Zamora', 'Zaragoza', 'Provincia'], dtype=object)

Aquí se usa LabelEncoder para pasar los valores categóricos de Provincia a numéricos, necesario para el entrenamiento del modelo:

In [12]:
df_fuel["num_provincia"] = le.fit_transform(df_fuel["Provincia"])

In [13]:
df_fuel.head()

,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro,num_provincia
0,2016-01-01,Albacete,Gasolina 95 E5,"0,482","1,155",0
1,2016-01-01,Albacete,Gasolina 98 E5,"0,552","1,278",0
2,2016-01-01,Albacete,Gasóleo A habitual,"0,441","0,993",0
3,2016-01-01,Albacete,Gasóleo Premium,"0,504","1,069",0
4,2016-01-01,Alicante/Alacant,Gasolina 95 E5,"0,483","1,157",1


In [14]:
df_fuel["num_provincia"].unique()

array([ 0,  1,  2,  3,  4, 52,  5,  6,  7,  8,  9, 16, 17, 10, 11, 12, 13,
       18, 14, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 30, 31,
       33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 38])

In [15]:
result = df_fuel.loc[df_fuel["num_provincia"] == 38]
result.head()

,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro,num_provincia
75762,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro,38
151318,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro,38
226874,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro,38
302430,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro,38
378193,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro,38


Puede parecer extraño pero esos valores se deben a que cuando se juntaron todos los archivos se añadió también la cabecera. Por lo tanto, las filas que sean de valor 38 las eliminamos.

In [16]:
df_fuel.drop(df_fuel[df_fuel["num_provincia"] == 38].index, inplace = True)

In [17]:
df_fuel["num_provincia"].unique()

array([ 0,  1,  2,  3,  4, 52,  5,  6,  7,  8,  9, 16, 17, 10, 11, 12, 13,
       18, 14, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 30, 31,
       33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51])

Se ha eliminado la etiqueta que sobra.

In [18]:
print(le.classes_)

['Albacete' 'Alicante/Alacant' 'Almería' 'Araba/Álava' 'Asturias'
 'Badajoz' 'Balears, Illes' 'Barcelona' 'Bizkaia' 'Burgos' 'Cantabria'
 'Castellón/Castelló' 'Ceuta' 'Ciudad Real' 'Coruña, A' 'Cuenca' 'Cáceres'
 'Cádiz' 'Córdoba' 'Gipuzkoa' 'Girona' 'Granada' 'Guadalajara' 'Huelva'
 'Huesca' 'Jaén' 'León' 'Lleida' 'Lugo' 'Madrid' 'Melilla' 'Murcia'
 'Málaga' 'Navarra' 'Ourense' 'Palencia' 'Palmas, Las' 'Pontevedra'
 'Provincia' 'Rioja, La' 'Salamanca' 'Santa Cruz de Tenerife' 'Segovia'
 'Sevilla' 'Soria' 'Tarragona' 'Teruel' 'Toledo' 'Valencia/València'
 'Valladolid' 'Zamora' 'Zaragoza' 'Ávila']


## Producto

De igual manera, hay que hacer lo mismo con Producto:  
En este caso uso un mapeo.

In [19]:
df_fuel["num_producto"] = df_fuel["Producto"].map({"Gasolina 95 E5":0, "Gasolina 98 E5":1, "Gasóleo A habitual":2, 
                                               "Gasóleo Premium":3})

In [20]:
df_fuel.head()

,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro,num_provincia,num_producto
0,2016-01-01,Albacete,Gasolina 95 E5,"0,482","1,155",0,0
1,2016-01-01,Albacete,Gasolina 98 E5,"0,552","1,278",0,1
2,2016-01-01,Albacete,Gasóleo A habitual,"0,441","0,993",0,2
3,2016-01-01,Albacete,Gasóleo Premium,"0,504","1,069",0,3
4,2016-01-01,Alicante/Alacant,Gasolina 95 E5,"0,483","1,157",1,0


In [21]:
df_fuel["num_producto"].unique()

array([0, 1, 2, 3], dtype=int64)

## Fechas

Se divide la fecha en tres columnas: año, mes, día.

In [22]:
df_fuel["Fecha Precio"] = pd.to_datetime(df_fuel["Fecha Precio"])

In [23]:
df_fuel["anno"] = df_fuel["Fecha Precio"].dt.year
df_fuel["mes"] = df_fuel["Fecha Precio"].dt.month
df_fuel["dia"] = df_fuel["Fecha Precio"].dt.day

In [24]:
df_fuel.head()

,Fecha Precio,Provincia,Producto,Promedio de Pai Diario CUBO €/litro,Promedio de Pvp Diario CUBO €/litro,num_provincia,num_producto,anno,mes,dia
0,2016-01-01,Albacete,Gasolina 95 E5,"0,482","1,155",0,0,2016,1,1
1,2016-01-01,Albacete,Gasolina 98 E5,"0,552","1,278",0,1,2016,1,1
2,2016-01-01,Albacete,Gasóleo A habitual,"0,441","0,993",0,2,2016,1,1
3,2016-01-01,Albacete,Gasóleo Premium,"0,504","1,069",0,3,2016,1,1
4,2016-01-01,Alicante/Alacant,Gasolina 95 E5,"0,483","1,157",1,0,2016,1,1


## Renombrado de columnas

Nos quedamos solo con las columnas que nos interesan y renombramos algunas de ellas:

In [25]:
df_fuel = df_fuel[["anno","mes","dia","num_provincia","num_producto","Promedio de Pai Diario CUBO €/litro","Promedio de Pvp Diario CUBO €/litro"]]
df_fuel.rename(columns={"Promedio de Pai Diario CUBO €/litro": "promedio_de_pai_diario_cubo", "Promedio de Pvp Diario CUBO €/litro": "promedio_de_pvp_diario_cubo"}, inplace = True)

In [26]:
df_fuel.head()

,anno,mes,dia,num_provincia,num_producto,promedio_de_pai_diario_cubo,promedio_de_pvp_diario_cubo
0,2016,1,1,0,0,"0,482","1,155"
1,2016,1,1,0,1,"0,552","1,278"
2,2016,1,1,0,2,"0,441","0,993"
3,2016,1,1,0,3,"0,504","1,069"
4,2016,1,1,1,0,"0,483","1,157"


## Pasar valores con comas a puntos

In [27]:
df_fuel["promedio_de_pai_diario_cubo"] = df_fuel["promedio_de_pai_diario_cubo"].str.replace(',', '.').astype(float)
df_fuel["promedio_de_pvp_diario_cubo"] = df_fuel["promedio_de_pvp_diario_cubo"].str.replace(',', '.').astype(float)

In [28]:
df_fuel.head()

,anno,mes,dia,num_provincia,num_producto,promedio_de_pai_diario_cubo,promedio_de_pvp_diario_cubo
0,2016,1,1,0,0,0.482,1.155
1,2016,1,1,0,1,0.552,1.278
2,2016,1,1,0,2,0.441,0.993
3,2016,1,1,0,3,0.504,1.069
4,2016,1,1,1,0,0.483,1.157


In [29]:
df_fuel.dtypes

anno                             int32
mes                              int32
dia                              int32
num_provincia                    int32
num_producto                     int64
promedio_de_pai_diario_cubo    float64
promedio_de_pvp_diario_cubo    float64
dtype: object

## Guardado del dataset transformado

In [30]:
dataset = "dataset.csv"
clean_csv_path = os.path.join(path, dataset)

df_fuel.to_csv(clean_csv_path, index=False)

# Entrenamiento

En caso de ser necesario (para no tener que arrancar todo lo anterior si cerramos el archivo) cargamos de nuevo el dataset:

In [31]:
df_fuel = pd.read_csv(clean_csv_path)

In [32]:
df_fuel.head()

,anno,mes,dia,num_provincia,num_producto,promedio_de_pai_diario_cubo,promedio_de_pvp_diario_cubo
0,2016,1,1,0,0,0.482,1.155
1,2016,1,1,0,1,0.552,1.278
2,2016,1,1,0,2,0.441,0.993
3,2016,1,1,0,3,0.504,1.069
4,2016,1,1,1,0,0.483,1.157


## Modelo de predicción sin impuestos (PAI):

In [33]:
def train_pai():
    X = df_fuel.drop(["promedio_de_pai_diario_cubo","promedio_de_pvp_diario_cubo"], axis=1)
    y = df_fuel["promedio_de_pai_diario_cubo"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    pai = DecisionTreeRegressor()
    pai_model = pai.fit(X_train.values, y_train)
    model_score = pai.score(X_test, y_test)

    print(f"La puntuación del modelo es de: {model_score} %")

    return pai_model

In [34]:
pai_model = train_pai()

La puntuación del modelo es de: 0.9991530236991897 %


e:\Programas\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


## Modelo de predicción con impuestos (PVP):

In [35]:
def train_pvp():
    X = df_fuel.drop(["promedio_de_pai_diario_cubo","promedio_de_pvp_diario_cubo"], axis=1)
    y = df_fuel["promedio_de_pvp_diario_cubo"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    pvp = DecisionTreeRegressor()
    pvp_model = pvp.fit(X_train.values, y_train)
    model_score = pvp.score(X_test, y_test)

    print(f"La puntuación del modelo es de: {model_score} %")

    return pvp_model

In [36]:
pvp_model = train_pvp()

La puntuación del modelo es de: 0.9994221218979142 %


e:\Programas\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


## Guardado de los modelos

In [37]:
directory = "model"
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

path = os.path.join(parent_dir, directory)

try:
    os.mkdir(path)
    print(f"Se ha creado el directorio {directory} en: {parent_dir}")
except OSError as error:
    print(f"El directorio {directory} en: {parent_dir} ya existe.")

El directorio model en: g:\Proyectos_Github\fuel-predict\docker ya existe.


In [38]:
def save_model(model, filename):
    directory = "model"
    path = os.path.join(parent_dir, directory)
    
    save_file = os.path.join(path, filename)

    with open(save_file, "wb") as f:
        pickle.dump(model, f)

In [39]:
save_model(pai_model, "pai_model.pkl")

In [40]:
save_model(pvp_model, "pvp_model.pkl")

Ejemplo de carga de modelo:

In [41]:
# with open('model.pkl', 'rb') as f:
#     clf2 = pickle.load(f)

# clf2.predict(X[0:1])